# Regression with Linear Algebra - Lab

## Introduction

In this lab, we shall apply regression analysis using simple matrix manipulations to fit a model to given data, and then predict new values for previously unseen data. We shall follow the approach highlighted in previous lesson where we used numpy to build the appropriate matrices and vectors and solve for the $\beta$ (unknown variables) vector. The beta vector will be used with test data to make new predictions. We shall also evaluate how good our model fit was. 

In order to make this experiment interesting. We shall use NumPy at every single stage of this experiment i.e. loading data, creating matrices, performing test train split, model fitting and evaluations.  

## Objectives

You will be able to:

* Use linear algebra to apply simple regression modeling in Python and NumPy only
* Apply train/test split using permutations in NumPy
* Use matrix algebra with inverses and dot products to calculate the beta
* Make predictions from the fitted model using previously unseen input features 
* Evaluate the fitted model by calculating the error between real and predicted values


First let's import necessary libraries 

In [1]:
import csv # for reading csv file
import numpy as np

## Dataset 

The dataset we will use for this experiment is "**Sales Prices in the City of Windsor, Canada**", something very similar to the Boston Housing dataset. This dataset contains a number of input (independent) variables, including area, number of bedrooms/bathrooms, facilities(AC/garage) etc. and an output (dependent) variable, **price**. We shall formulate a linear algebra problem to find linear mappings from input to out features using the equation provided in the previous lesson. 

This will allow us to find a relationship between house features and house price for the given data, allowing us to find unknown prices for houses, given the input features.  

A description of dataset and included features is available at [THIS LINK](https://rdrr.io/cran/Ecdat/man/Housing.html)

In your repo, the dataset is available as `windsor_housing.csv` containing following variables:

there are 11 input features (first 11 columns):

	lotsize	bedrooms	bathrms	stories	driveway	recroom	fullbase	gashw	airco	garagepl	prefarea

and 1 output feature i.e. **price** (12th column). 

The focus of this lab is not really answering a preset analytical question, but to learn how we can perform a regression experiment, similar to one we performed in statsmodels, using mathematical manipulations. So we we wont be using any Pandas or statsmodels goodness here. The key objectives here are to a) understand regression with matrix algebra, and b) Mastery in NumPy scientific computation. 

## Stage 1: Prepare Data for Modeling 

Let's give you a head start by importing the dataset. We shall perform following steps to get the data ready for analysis:

* Initialize an empty list `data` for loading data
* Read the csv file containing complete (raw) `windsor_housing.csv`. [Use `csv.reader()` for loading data.](https://docs.python.org/3/library/csv.html). Store this in `data` one row at a time.

* Drop the first row of csv file as it contains the names of variables (header) which won't be used during analysis (keeping this will cause errors as it contains text values).

* Append a column of all 1s to the data (bias) as the first column

* Convert `data` to a numpy array and inspect first few rows 

NOTE: `read.csv()` would read the csv as a text file, so we must convert the contents to float at some stage. 

In [2]:
data

NameError: name 'data' is not defined

In [3]:
# Your Code here
data = []
with open("windsor_housing.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append([1] + row)

data = np.array(data[1:])

data = data.astype(float)

# First 5 rows of raw data 

# array([[1.00e+00, 5.85e+03, 3.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
#         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
#         4.20e+04],
#        [1.00e+00, 4.00e+03, 2.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
#         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
#         3.85e+04],
#        [1.00e+00, 3.06e+03, 3.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
#         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
#         4.95e+04],
#        [1.00e+00, 6.65e+03, 3.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
#         1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
#         6.05e+04],
#        [1.00e+00, 6.36e+03, 2.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
#         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
#         6.10e+04]])

## Step 2: Perform a 80/20 test train Split

Explore NumPy's official documentation to manually split a dataset using `numpy.random.shuffle()`,  `numpy.random.permutations()` or using simple resampling method. 
* Perform a **RANDOM** 80/20 split on data using a method of your choice , in NumPy using one of the methods above. 
* Create x_test, y_test, x_train and y_train arrays from the split data.
* Inspect the contents to see if the split performed as expected. 

In [4]:
# Your code here 
import numpy as np

eighty = round(len(data) * .80)

np.random.seed(345)
np.random.shuffle(data)
X = data[:,:-1]
y = data[:,-1]


X_train = X[:eighty]
X_test = X[eighty:]
y_train = y[:eighty]
y_test = y[eighty:]
print(data.shape)
print(X.shape, y.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Split results

# Raw data Shape:  (546, 13)
# Train/Test Split: (437, 13) (109, 13)
# x_train, y_train, x_test, y_test: (437, 12) (437,) (109, 12) (109,)

(546, 13)
(546, 12) (546,)
(437, 12) (109, 12) (437,) (109,)


## Step 3: Calculate the `beta` 

With our X and y in place, We can now compute our beta values with x_train and y_train as:
#### $\beta$ = (x_train<sup>T</sup> . x_train)<sup>-1</sup> . x_train<sup>T</sup> . y_train 

* Using numpy operations (transpose, inverse) that we saw earlier, compute the above equation in steps.
* Print your beta values

In [5]:
# Your code here 
def beta(x, y):
    return np.linalg.inv((x.T.dot(x))).dot(x.T.dot(y))

b = beta(X_train, y_train)
print(b)


# Calculated beta values

# [-3.07118956e+03  2.13543921e+00  4.04283395e+03  1.33559881e+04
#   5.75279185e+03  7.82810082e+03  3.73584043e+03  6.51098935e+03
#   1.28802060e+04  1.09853850e+04  6.14947126e+03  1.05813305e+04]

[-4.06367243e+03  3.30133737e+00  2.14584345e+03  1.45222451e+04
  6.70414379e+03  6.08424917e+03  5.52605797e+03  5.08510135e+03
  1.21178683e+04  1.30337930e+04  4.79416970e+03  9.78500754e+03]


## Step 4: Make Predictions
Great , we now have a set of coefficients that describe the linear mappings between X and y. We can now use the calculated beta values  with the test datasets that we left out to calculate y predictions. 
For this we need to perform the following tasks:

Now we shall all features in each row in turn and multiply it with the beta computed above. The result will give a prediction for each row which we can append to a new array of predictions.

#### $\hat{y}$ = x.$\beta$ = $\beta$<sub>0</sub> + $\beta$<sub>1</sub> . x<sub>1</sub> + $\beta$<sub>2</sub> . x<sub>2</sub> + .. + $\beta$<sub>m</sub> . x<sub>m</sub>


* Create new empty list (y_pred) for saving predictions.
* For each row of x_test, take the dot product of the row with beta to calculate the prediction for that row.
* Append the predictions to y_pred.
* Print the new set of predictions.

In [6]:
# Your code here 
y_pred = []
for row in X_test:
    y_pred.append(row.dot(b))
y_pred = np.array(y_pred)
print(y_pred)

[ 56826.34704873  63778.03486821  69058.78746024  54388.15897585
  36897.94400012  52958.95030765  49560.97590688  56041.25294995
  51377.75326302  51611.82401292  46480.94374174  54388.15897585
  86446.32394506  49558.54698015  89188.44150272  61855.02568892
  51460.96256442  44146.80845512 100299.45678855  79898.63456372
  49625.61552992 115173.30759122  45934.74618556  74682.49108201
  78393.51465154  72633.26342186  59959.15979174  31358.41549395
  98253.19522475  92107.51458959  40876.05553158  40526.11377031
  82586.18727622  68997.92066266  87029.29842265  65168.36931287
  52071.03411083  42155.72303159  46363.48023878  52928.31418893
  50221.26921686  56361.70505426  40083.73456266  88803.34736979
  71448.22226127  55504.45081203  79807.18605905  86317.61687448
 100514.42976425  43220.00506464  67050.97488101  54227.40473433
  40208.40273777  53755.43215881  56920.52931639  73457.15503958
  56373.78881531  76215.56728848  40703.60334335  59680.94460567
  63554.98120368  45887.7

## Step 5: Evaluate Model 

### Visualize Actual vs. Predicted
This is exciting, so now our model can use the beta value to predict the price of houses given the input features. Let's plot these predictions against the actual values in y_test to see how much our model deviates. 

In [7]:
# Plot predicted and actual values as line plots
import matplotlib.pyplot as plt

plt.figure(figsize=[15, 10])

plt.plot(y_pred, linestyle='-', marker='o', markerfacecolor="blue", label='predictions', c="green")
plt.plot(y_test, linestyle='-', marker='o', label='actual values', c="magenta")
plt.title('Actual vs. predicted values')
plt.legend()
plt.show()


<Figure size 1500x1000 with 1 Axes>

![](diff.png)

This doesn't look so bad, does it ? Our model, although isn't perfect at this stage, is making a good attempt to predict house prices although a few prediction seem a bit out. There could a number of reasons for this. Let's try to dig a bit deeper to check model's predictive abilities by comparing these prediction with actual values of y_test individually. That will help us calculate the RMSE value (Root Mean Squared Error) for our model. 
### Root Mean Squared Error
Here is the formula for this again. 

![](rmse.jpg)


* Initialize an empty array `err`.
* for each row in y_test and y_pred, take the squared difference and append error for each row in err array. 
* Calculate RMSE from `err` using the formula shown above. 

In [8]:
# Calculate RMSE
err = (y_test - y_pred) ** 2

RMSE = np.sqrt(sum(err)/len(y_test))

RMSE2 = np.sqrt(err.mean())
print(round(RMSE, 5) == round(RMSE2, 5))

print(RMSE)

# Due to random split, your answers may vary 

# RMSE = 16401.913562758735

True
15416.819783458828


### Normalized Root Mean Squared Error
The above error is clearly in terms of the dependant variable i.e. the final house price. We can also use a normlized mean squared error in case of multiple regression which can be calculated from RMSE using following formula:

* Calculate normalized Root Mean Squared Error

<img src="nrmse.png" width=300>

In [9]:
# Calculate NRMSE

NRMSE = RMSE/((y_train).max()-y_train.min())

NRMSE

# Due to random split, your answers may vary 

# 0.09940553674399233

0.09343527141490199

SO there it is. A complete multiple regression analysis using nothing but numpy. Having good programming skills in numpy would allow to dig deeper into analytical algorithms in machine learning and deep learning. Using matrix multiplication techniques we saw here, we can easily build a whole neural network from scratch. 

## Level up - Optional 

* Calculated the R_squared and adjusted R_squared for above experiment. 
* Plot the residuals (similar to statsmodels) and comment on the variance and heteroscedascticity. 
* Run the experiment in statsmodels and compare the performance of both approaches in terms of computational cost.

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [20]:
lin = LinearRegression()
lin.fit(X_train, y_train)
y_pred_lin = lin.predict(X_test)
print(mean_squared_error(y_test, y_pred_lin)**.5)
r2_score(y_test, y_pred)

15416.819783458835


0.586921620498517

In [12]:
reg = sm.OLS(y_train, X_train)
results = reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     84.16
Date:                Thu, 14 Mar 2019   Prob (F-statistic):           2.12e-99
Time:                        13:10:44   Log-Likelihood:                -4829.2
No. Observations:                 437   AIC:                             9682.
Df Residuals:                     425   BIC:                             9731.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4063.6724   3729.377     -1.090      0.276   -1.14e+04    3266.648
x1             3.3013      0.379      8.706      0.000       2.556       4.047
x2          2145.8435   1144.646      1.875      0.062    -104.028    4395.715
x3          1.452e+04   1655.662      8.771      0.000    1.13e+04    1.78e+04
x4          6704.1438   1012.462      6.622      0.000    4714.088    8694.199
x5          6084.2492   2301.319      2.644      0.009    1560.865    1.06e+04
x6          5526.0580   2055.497      2.688      0.007    1485.853    9566.263
x7          5085.1014   1772.726      2.869      0.004    1600.700    8569.502
x8          1.212e+04   3444.731      3.518      0.000    5347.038    1.89e+04
x9          1.303e+04   1721.211      7.572      0.000    9650.647    1.64e+04
x10         4794.1697    934.874      5.128      0.000    2956.617    6631.722
x11         9785.0075   1872.681      5.225      0.000    6104.137    1.35e+04
==============================================================================
Omnibus:                       75.873   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              205.503
Skew:                           0.836   Prob(JB):                     2.37e-45
Kurtosis:                       5.913   Cond. No.                     3.05e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [62]:
SSE = sum(err)
SST = sum((y_pred - y_train.mean())**2)

R2 = 1 - (SSE/SST)
print(R2)

nev = 11 # number of explanatory variables aka the independent variables not including the constant
adj_R2 = 1 - ((1-R2)*len(y_pred) -1)/(len(y_pred) - nev - 1)
print(adj_R2)

0.45549313307766903
0.3984407371697518


## Summary

So there we have it. A predictive model for predicting house prices in a given dataset. Remember this is a very naive implementation of regression modeling. The purpose here was to get an introduction to the applications of linear algebra into machine learning and predictive analysis. We still have a number of shortcomings in our modeling approach and we can further apply a number of data modeling techniques to improve this model. 